# 循環神經網絡

在上一個模組中，我們探討了文本的豐富語義表示。我們使用的架構能夠捕捉句子中詞語的聚合意義，但它並未考慮詞語的**順序**，因為嵌入後的聚合操作會移除原始文本中的這些信息。由於這些模型無法表示詞語的排列順序，因此它們無法解決更複雜或更具歧義的任務，例如文本生成或問題回答。

為了捕捉文本序列的意義，我們將使用一種名為**循環神經網絡**（Recurrent Neural Network，簡稱 RNN）的神經網絡架構。在使用 RNN 時，我們會將句子逐個標記傳遞給網絡，網絡會生成某種**狀態**，然後我們將該狀態與下一個標記一起再次傳遞給網絡。

![圖片展示了一個循環神經網絡生成的示例。](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

給定標記的輸入序列 $X_0,\dots,X_n$，RNN 會創建一系列神經網絡塊，並通過反向傳播對這個序列進行端到端的訓練。每個網絡塊以 $(X_i,S_i)$ 作為輸入，並生成 $S_{i+1}$ 作為結果。最終狀態 $S_n$ 或輸出 $Y_n$ 會進入線性分類器以生成結果。所有網絡塊共享相同的權重，並通過一次反向傳播進行端到端訓練。

> 上圖展示了循環神經網絡的展開形式（左側）和更緊湊的循環表示形式（右側）。需要注意的是，所有 RNN 單元都具有相同的**可共享權重**。

由於狀態向量 $S_0,\dots,S_n$ 是通過網絡傳遞的，RNN 能夠學習詞語之間的順序依賴。例如，當某個序列中出現詞語 *not* 時，它可以學習在狀態向量中否定某些元素。

在內部，每個 RNN 單元包含兩個權重矩陣：$W_H$ 和 $W_I$，以及偏置 $b$。在每個 RNN 步驟中，給定輸入 $X_i$ 和輸入狀態 $S_i$，輸出狀態的計算公式為 $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$，其中 $f$ 是激活函數（通常為 $\tanh$）。

> 對於像文本生成（我們會在下一單元中探討）或機器翻譯這類問題，我們還希望在每個 RNN 步驟中獲得某些輸出值。在這種情況下，還會有另一個矩陣 $W_O$，輸出值的計算公式為 $Y_i=f(W_O\times S_i+b_O)$。

現在讓我們看看循環神經網絡如何幫助我們對新聞數據集進行分類。

> 在沙盒環境中，我們需要運行以下單元格以確保所需的庫已安裝並且數據已預取。如果您在本地運行，可以跳過以下單元格。


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

在訓練大型模型時，GPU 記憶體分配可能會成為一個問題。我們可能需要嘗試不同的迷你批次大小，以確保數據能夠適配 GPU 記憶體，同時訓練速度也足夠快。如果你在自己的 GPU 機器上運行此代碼，可以嘗試調整迷你批次大小來加快訓練速度。

> **Note**: 某些版本的 NVidia 驅動程式已知在訓練模型後不會釋放記憶體。我們在這個筆記本中運行了幾個範例，這可能會導致某些配置下記憶體耗盡，特別是當你在同一個筆記本中進行自己的實驗時。如果在開始訓練模型時遇到一些奇怪的錯誤，你可能需要重新啟動筆記本的內核。


In [3]:
batch_size = 16
embed_size = 64

## 簡單 RNN 分類器

在簡單 RNN 的情況下，每個循環單元都是一個簡單的線性網絡，它接收輸入向量和狀態向量，並生成新的狀態向量。在 Keras 中，這可以用 `SimpleRNN` 層來表示。

雖然我們可以直接將 one-hot 編碼的標記傳遞給 RNN 層，但這並不是一個好主意，因為它們的維度太高。因此，我們會使用嵌入層來降低詞向量的維度，接著是 RNN 層，最後是一個 `Dense` 分類器。

> **注意**：在維度不是很高的情況下，例如使用字符級標記化時，直接將 one-hot 編碼的標記傳遞給 RNN 單元可能是合理的。


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **注意：** 我們在這裡使用未經訓練的嵌入層以簡化操作，但為了獲得更好的效果，我們可以使用 Word2Vec 預訓練的嵌入層，如前一單元所述。這會是一個很好的練習，讓你嘗試將此程式碼改為使用預訓練的嵌入層。

現在讓我們開始訓練 RNN。一般來說，RNN 的訓練相當困難，因為當 RNN 單元沿著序列長度展開時，反向傳播所涉及的層數會非常多。因此，我們需要選擇較小的學習率，並在更大的數據集上訓練網絡以獲得良好的結果。這可能需要相當長的時間，因此建議使用 GPU。

為了加快速度，我們將僅使用新聞標題來訓練 RNN 模型，省略描述部分。你可以嘗試加入描述進行訓練，看看是否能讓模型成功訓練。


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **注意**，由於我們僅針對新聞標題進行訓練，因此準確性可能會較低。


## 重溫變數序列

記住，`TextVectorization` 層會自動在小批量中用填充標記來填充可變長度的序列。然而，這些填充標記也會參與訓練，並可能使模型的收斂變得更複雜。

我們可以採取幾種方法來減少填充的數量。其中一種方法是根據序列長度重新排列數據集，並按大小分組所有序列。這可以使用 `tf.data.experimental.bucket_by_sequence_length` 函數來完成（請參閱[文件](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)）。

另一種方法是使用**遮罩**。在 Keras 中，某些層支持額外的輸入，該輸入顯示哪些標記應在訓練時被考慮。要將遮罩整合到我們的模型中，我們可以選擇添加一個單獨的 `Masking` 層（[文件](https://keras.io/api/layers/core_layers/masking/)），或者在我們的 `Embedding` 層中指定 `mask_zero=True` 參數。

> **Note**: 完成整個數據集的一個訓練週期大約需要 5 分鐘。如果你失去耐心，可以隨時中斷訓練。你也可以通過在 `ds_train` 和 `ds_test` 數據集後添加 `.take(...)` 子句來限制用於訓練的數據量。


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


現在我們使用遮罩技術，可以在整個標題和描述的數據集上訓練模型。

> **注意**：你有沒有注意到，我們一直在使用基於新聞標題訓練的向量化工具，而不是整篇文章的內容？這可能會導致某些詞元被忽略，因此重新訓練向量化工具會更好。不過，這可能只會帶來非常小的影響，所以為了簡化流程，我們會繼續使用之前預訓練的向量化工具。


## LSTM：長短期記憶

RNNs 的主要問題之一是 **梯度消失**。RNNs 可以非常長，在反向傳播過程中可能難以將梯度傳遞回網絡的第一層。當出現這種情況時，網絡無法學習遠距離的標記之間的關係。解決這個問題的一種方法是通過使用 **閘門** 引入 **顯式狀態管理**。最常見的引入閘門的架構是 **長短期記憶**（LSTM）和 **門控循環單元**（GRU）。我們在這裡將討論 LSTM。

![顯示長短期記憶單元示例的圖片](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM 網絡的組織方式與 RNN 類似，但有兩個狀態會從層到層傳遞：實際狀態 $c$ 和隱藏向量 $h$。在每個單元中，隱藏向量 $h_{t-1}$ 與輸入 $x_t$ 結合，並共同控制狀態 $c_t$ 和輸出 $h_{t}$ 的變化，這是通過 **閘門** 完成的。每個閘門都有 sigmoid 激活函數（輸出範圍為 $[0,1]$），可以被視為在與狀態向量相乘時的位掩碼。LSTM 包含以下閘門（如上圖所示，從左到右）：
* **遺忘閘門**：決定向量 $c_{t-1}$ 的哪些部分需要遺忘，哪些需要保留。
* **輸入閘門**：決定來自輸入向量和前一隱藏向量的信息有多少應該被整合到狀態向量中。
* **輸出閘門**：接收新的狀態向量並決定其哪些部分將用於生成新的隱藏向量 $h_t$。

狀態 $c$ 的組件可以被視為可以開啟或關閉的標誌。例如，當我們在序列中遇到名字 *Alice* 時，我們猜測它指的是一位女性，並在狀態中啟動表示句子中有女性名詞的標誌。當我們進一步遇到 *and Tom* 這些詞時，我們會啟動表示句子中有複數名詞的標誌。因此，通過操控狀態，我們可以追蹤句子的語法屬性。

> **注意**：這裡有一個很棒的資源可以幫助理解 LSTM 的內部結構：[理解 LSTM 網絡](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 作者是 Christopher Olah。

雖然 LSTM 單元的內部結構看起來可能很複雜，但 Keras 將這些實現隱藏在 `LSTM` 層中，因此在上面的示例中，我們唯一需要做的就是替換循環層：


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## 雙向和多層 RNN

在我們之前的例子中，循環神經網絡（RNN）都是從序列的開頭運行到結尾。這對我們來說很自然，因為它符合我們閱讀或聆聽語音的方向。然而，對於需要隨機訪問輸入序列的情境來說，讓循環計算在兩個方向上運行會更有意義。允許雙向計算的 RNN 被稱為 **雙向 RNN**，可以通過將循環層包裹在一個特殊的 `Bidirectional` 層中來創建。

> **Note**: `Bidirectional` 層會在內部生成該層的兩個副本，並將其中一個副本的 `go_backwards` 屬性設置為 `True`，使其沿著序列的相反方向運行。

無論是單向還是雙向的循環網絡，都能捕捉序列中的模式，並將其存儲到狀態向量中或作為輸出返回。與卷積網絡類似，我們可以在第一個循環層之後再構建另一個循環層，以捕捉更高層次的模式，這些模式是由第一層提取的低層次模式構建而成的。這引出了 **多層 RNN** 的概念，它由兩層或更多層循環網絡組成，其中前一層的輸出作為下一層的輸入。

![顯示多層長短期記憶 RNN 的圖片](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*圖片來源於 [這篇精彩的文章](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3)，作者是 Fernando López。*

Keras 讓構建這些網絡變得非常簡單，因為你只需要在模型中添加更多的循環層。對於最後一層以外的所有層，我們需要指定 `return_sequences=True` 參數，因為我們需要該層返回所有的中間狀態，而不僅僅是循環計算的最終狀態。

現在，我們來為分類問題構建一個雙層雙向 LSTM。

> **Note** 這段代碼執行起來可能需要較長的時間，但它能給我們帶來目前為止最高的準確率。所以也許值得等待並看看結果。


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs 用於其他任務

到目前為止，我們主要集中在使用 RNNs 來對文本序列進行分類。但它們還能處理更多任務，例如文本生成和機器翻譯——我們會在下一單元探討這些任務。



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原文文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
